In [1]:
# %pip install pyspark

In [61]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import broadcast, substring, col, avg, monotonically_increasing_id, lit, when
from pyspark.sql.types import StructType, StructField, StringType

In [3]:
sc = SparkContext(appName="Simple App")
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/13 07:27:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_schema=StructType([
    StructField('tourney_id', StringType(), True),
    StructField('tourney_name', StringType(), True),
    StructField('surface', StringType(), True),
    StructField('draw_size', StringType(), True),
    StructField('tourney_level', StringType(), True),
    StructField('tourney_date', StringType(), True),
    StructField('match_num', StringType(), True),
    StructField('winner_id', StringType(), True),
    StructField('winner_seed', StringType(), True),
    StructField('winner_entry', StringType(), True),
    StructField('winner_name', StringType(), True),
    StructField('winner_hand', StringType(), True),
    StructField('winner_ht', StringType(), True),
    StructField('winner_ioc', StringType(), True),
    StructField('winner_age', StringType(), True),
    StructField('loser_id', StringType(), True),
    StructField('loser_seed', StringType(), True),
    StructField('loser_entry', StringType(), True),
    StructField('loser_name', StringType(), True),
    StructField('loser_hand', StringType(), True),
    StructField('loser_ht', StringType(), True),
    StructField('loser_ioc', StringType(), True),
    StructField('loser_age', StringType(), True),
    StructField('score', StringType(), True),
    StructField('best_of', StringType(), True),
    StructField('round', StringType(), True),
    StructField('minutes', StringType(), True),
    StructField('w_ace', StringType(), True),
    StructField('w_df', StringType(), True),
    StructField('w_svpt', StringType(), True),
    StructField('w_1stIn', StringType(), True),
    StructField('w_1stWon', StringType(), True),
    StructField('w_2ndWon', StringType(), True),
    StructField('w_SvGms', StringType(), True),
    StructField('w_bpSaved', StringType(), True),
    StructField('w_bpFaced', StringType(), True),
    StructField('l_ace', StringType(), True),
    StructField('l_df', StringType(), True),
    StructField('l_svpt', StringType(), True),
    StructField('l_1stIn', StringType(), True),
    StructField('l_1stWon', StringType(), True),
    StructField('l_2ndWon', StringType(), True),
    StructField('l_SvGms', StringType(), True),
    StructField('l_bpSaved', StringType(), True),
    StructField('l_bpFaced', StringType(), True),
    StructField('winner_rank', StringType(), True),
    StructField('winner_rank_points', StringType(), True),
    StructField('loser_rank', StringType(), True),
    StructField('loser_rank_points', StringType(), True)
])

df = spark.read.schema(df_schema).csv('../files/atp/atp_matches_*', header=True)

In [5]:
# Carlos Alcaraz ID 207989
alcaraz_df = df.select(
    col('tourney_name'), 
    col('surface'),
    col('draw_size'),
    col('tourney_level'), 
    col('tourney_date'),
    col('winner_id'),
    col('winner_seed'),
    col('winner_name'),
    col('winner_hand'),
    col('winner_ioc'),
    col('winner_age'),
    col('loser_id'),
    col('loser_seed'),
    col('loser_name'),
    col('loser_hand'),
    col('loser_ioc'),
    col('loser_age'),
    col('score'),
    col('best_of'),
    col('round'),
    col('minutes'),
    col('w_ace'),
    col('w_df'),
    col('w_svpt'),
    col('w_1stIn'),
    col('w_1stWon'),
    col('w_2ndWon'),
    col('w_SvGms'),
    col('w_bpSaved'),
    col('w_bpFaced'),
    col('l_ace'),
    col('l_df'),
    col('l_svpt'),
    col('l_1stIn'),
    col('l_1stWon'),
    col('l_2ndWon'),
    col('l_SvGms'),
    col('l_bpSaved'),
    col('l_bpFaced'),
    col('winner_rank'),
    col('winner_rank_points'),
    col('loser_rank'),
    col('loser_rank_points')
    ).where((col('winner_id') == '207989') | (col('loser_id') == '207989')).withColumn('match_id', monotonically_increasing_id()).cache()

24/06/13 07:27:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [39]:
alcaraz_matches_over_the_years = alcaraz_df.groupby(col('tourney_date')[0:4].alias('Year')).count().orderBy(col('Year')).withColumnRenamed('count', 'Matches')
alcaraz_wins_over_the_years = alcaraz_df.where(col('winner_id') == '207989').groupby(col('tourney_date')[0:4].alias('Year')).count().orderBy(col('Year')).withColumnRenamed('count', 'Wins')
alcaraz_losses_over_the_years = alcaraz_df.where(col('loser_id') == '207989').groupby(col('tourney_date')[0:4].alias('Year')).count().orderBy(col('Year')).withColumnRenamed('count', 'Losses')

alcaraz_matches_wins_losses_over_the_years = alcaraz_matches_over_the_years.join(alcaraz_wins_over_the_years,['Year'])
alcaraz_matches_wins_losses_over_the_years = alcaraz_matches_wins_losses_over_the_years.join(alcaraz_losses_over_the_years,['Year']).orderBy(col('Year'))

alcaraz_matches_wins_losses_percentage_over_the_years = alcaraz_matches_wins_losses_over_the_years.withColumn('%', (col('Wins')/ ((col('Wins')+col('Losses'))) * 100).cast('int') )
alcaraz_matches_wins_losses_percentage_over_the_years.show()

+----+-------+----+------+---+
|Year|Matches|Wins|Losses|  %|
+----+-------+----+------+---+
|2020|      2|   1|     1| 50|
|2021|     49|  32|    17| 65|
|2022|     70|  57|    13| 81|
|2023|     77|  65|    12| 84|
|2024|     23|  18|     5| 78|
+----+-------+----+------+---+



In [45]:
alcaraz_matches_each_surface = alcaraz_df.groupby(col('Surface')).count().withColumnRenamed('count', 'Matches')
alcaraz_wins_each_surface = alcaraz_df.where(col('winner_id') == '207989').groupby(col('Surface')).count().withColumnRenamed('count', 'Wins')
alcaraz_losses_each_surface = alcaraz_df.where(col('loser_id') == '207989').groupby(col('Surface')).count().withColumnRenamed('count', 'Losses')

alcaraz_matches_wins_losses_each_surface = alcaraz_matches_each_surface.join(alcaraz_wins_each_surface,['Surface'])
alcaraz_matches_wins_losses_each_surface = alcaraz_matches_wins_losses_each_surface.join(alcaraz_losses_each_surface,['Surface'])

alcaraz_matches_wins_losses_percentage_each_surface = alcaraz_matches_wins_losses_each_surface.withColumn('%', (col('Wins')/ ((col('Wins')+col('Losses'))) * 100).cast('int') )

alcaraz_matches_wins_losses_percentage_each_surface.show()

+-------+-------+----+------+---+
|Surface|Matches|Wins|Losses|  %|
+-------+-------+----+------+---+
|   Clay|     86|  69|    17| 80|
|   Hard|    117|  88|    29| 75|
|  Grass|     18|  16|     2| 88|
+-------+-------+----+------+---+



In [66]:
alcaraz_titles_by_level = alcaraz_df.where((col('winner_id') == '207989') & (col('round') == 'F')).groupBy(col('tourney_level').alias('Tourney Level')).count()
alcaraz_titles_by_level = alcaraz_titles_by_level.select(when(col('Tourney Level') == 'M', 'Masters').when(col('Tourney Level') == 'A', 'ATP').when(col('Tourney Level') == 'G', 'Grand Slams').alias('Level'),col('count').alias('Titles'))
alcaraz_titles_by_level.show()

+-----------+------+
|      Level|Titles|
+-----------+------+
|    Masters|     5|
|        ATP|     7|
|Grand Slams|     2|
+-----------+------+



In [70]:
alcaraz_titles_by_name_level_year = alcaraz_df.select(col('tourney_name').alias('Tourney Name'),when(col('tourney_level') == 'M', 'Masters').when(col('tourney_level') == 'A', 'ATP').when(col('tourney_level') == 'G', 'Grand Slams').alias('Level'), col('tourney_date')[0:4].alias('Year')).where((col('winner_id') == '207989') & (col('round') == 'F')).orderBy(col('Year'))
alcaraz_titles_by_name_level_year.show()

+--------------------+-----------+----+
|        Tourney Name|      Level|Year|
+--------------------+-----------+----+
|                Umag|        ATP|2021|
|      NextGen Finals|        ATP|2021|
|      Rio de Janeiro|        ATP|2022|
|       Miami Masters|    Masters|2022|
|           Barcelona|        ATP|2022|
|      Madrid Masters|    Masters|2022|
|             Us Open|Grand Slams|2022|
|        Buenos Aires|        ATP|2023|
|Indian Wells Masters|    Masters|2023|
|           Barcelona|        ATP|2023|
|      Madrid Masters|    Masters|2023|
|        Queen's Club|        ATP|2023|
|           Wimbledon|Grand Slams|2023|
|Indian Wells Masters|    Masters|2024|
+--------------------+-----------+----+

